In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

%cd 'gdrive/My Drive/licenta/car_model_od'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/licenta/car_model_od'
/content/gdrive/My Drive/licenta/car_model_od


In [3]:
# %pip install numpy==1.17

In [4]:
from datasets import get_test_loader_voc
from model import construct_model_detection, construct_model_classification

from IPython.display import display # to display images

import argparse
import torch
import json
import os
import time
import numpy as np

from PIL import Image
import cv2 as cv

from torchvision.ops import nms
import torchvision.transforms as transforms
import torch.nn as nn

In [5]:
def load_weight(model, path, device):
    sd = torch.load(os.path.join(path, 'best.pth'), map_location=device)
    model.load_state_dict(sd['model'])
    epoch = sd['epoch']

    print('Loaded model from epoch %d\n' % (epoch))

In [6]:
def load_model_detection(path, num_classes, device):
    print('Loading model for car detection....')

    with open(os.path.join(path, 'config.json')) as json_file:
        config = json.load(json_file)
        
    best_path = os.path.join(path, 'best.pth')
    checkpoint = torch.load(best_path, map_location=device)
    model_state_dict = checkpoint['model']

    model_od = construct_model_detection(config, num_classes)
    model_od.load_state_dict(model_state_dict)
    model_od.to(device)
    model_od.eval()

    return model_od, config['imgsize']
    
def load_model_classification(path, num_classes, device):
    print('Loading model for car classification....')

    with open(os.path.join(path, 'config.json')) as json_file:
        config = json.load(json_file)
        
    best_path = os.path.join(path, 'best.pth')
    checkpoint = torch.load(best_path, map_location=device)
    model_state_dict = checkpoint['model']

    model_cl = construct_model_classification(config, num_classes)
    model_cl.load_state_dict(model_state_dict)
    model_cl.to(device)
    model_cl.eval()

    return model_cl, config['imgsize']

In [7]:
@torch.no_grad()
def get_prediction_detection(model_od, image, img_size, threshold=0.6):
    
    # print('forward model detection')
    pred = model_od(image) # Pass the image to the model

    # pred_class = [CATEGORY_NAMES_CAR[i] for i in list(pred[0]['labels'])] # Get the Prediction Class
    # pred_boxes = [[box[0] / img_size[0] * original_width, box[1] / img_size[1] * original_height, box[2] / img_size[0] * original_width, box[3] / img_size[1] * original_height]
    #                 for box in list(pred[0]['boxes'].detach().cpu().numpy())] # Bounding boxes

    pred_class = [label.item() for label in list(pred[0]['labels'])] # Get the Prediction Class
    pred_boxes = [[box[0], box[1], box[2], box[3]]
                    for box in list(pred[0]['boxes'].detach().cpu().numpy())] # Bounding boxes

    pred_score = list(pred[0]['scores'].detach().cpu().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x > threshold] # Get list of index with score greater than threshold.

    pred_boxes = [pred_boxes[t] for t in pred_t]
    pred_class = [pred_class[t] for t in pred_t]
    pred_score = [pred_score[t] for t in pred_t]

    keep = nms(boxes=torch.FloatTensor(pred_boxes), scores=torch.FloatTensor(pred_score), iou_threshold=0.1)
    keep_boxes = [pred_boxes[i] for i in keep]
    keep_class = [pred_class[i] for i in keep]
    keep_score = [pred_score[i] for i in keep]

    return keep_boxes, keep_class, keep_score

def transform_image(image, img_size):
    mean_nums = [0.485, 0.456, 0.406]
    std_nums = [0.229, 0.224, 0.225]

    inference_transform = transforms.Compose([transforms.Resize(img_size),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean_nums, std_nums)])

    return inference_transform(image).unsqueeze(0)

@torch.no_grad()
def get_prediction_classification(model_cl, img, img_size, device):
    # print('classification at {}px'.format(img_size))
    
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    image = Image.fromarray(img)
    # image = img

    # image = torch.from_numpy(img).float()
    image = transform_image(image, img_size=img_size)
    # Move to default device
    image = image.to(device)
    
    # print('forward model classification')
    pred = model_cl(image) # Pass the image to the model

    probabilities = nn.Softmax(dim=1)(pred)
    scores, ids = torch.topk(probabilities, 1)
    scores, ids = scores.cpu(), ids.cpu()

    return ids[0].item(), scores[0].item()

In [8]:
def main(args):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model_path_od = args.path_od
    model_path_cl = args.path_cl
    config_dataset = args.config_dataset
    threshold = args.threshold

    if config_dataset is not None:
        with open(config_dataset) as json_file:
            config_od = json.load(json_file)

    else:
      with open(os.path.join(model_path_od, 'config.json')) as json_file:
          config_od = json.load(json_file)

    test_dataloader = get_test_loader_voc(config_od)

    num_classes_od = len(test_dataloader.dataset.labels.keys())

    model_cl = None

    groundtruths_folder_path = os.path.join(model_path_od, 'groundtruths')
    detections_folder_path = os.path.join(model_path_od, 'detections')	

    if config_od['train_only_car']:
        num_classes_od = 2

    if model_path_cl is not None:
        model_cl, img_size_cl = load_model_classification(model_path_cl, num_classes_od, device)
        num_classes_od = 2

        groundtruths_folder_path = os.path.join(model_path_cl, 'groundtruths')
        detections_folder_path = os.path.join(model_path_cl, 'detections')	

    model_od, img_size_od = load_model_detection(model_path_od, num_classes_od, device)

    os.makedirs(groundtruths_folder_path, exist_ok=True)
    os.makedirs(detections_folder_path, exist_ok=True)

    i = 0

    with torch.no_grad():
        for image, target in test_dataloader:
            i += 1
            print(f'{i}/{len(test_dataloader)}')

            image = list(image.to(device) for image in image)

            annotations_list = target[0]
            # number of objects in the image
            num_objs = len(annotations_list['boxes'])

            gt_class = [i for i in list(annotations_list['labels'].detach().cpu().numpy())]
            gt_boxes = [[box[0], box[1], box[2], box[3]] 
                            for box in list(annotations_list['boxes'].detach().cpu().numpy())]

            filename = annotations_list['filename'].split('.')[0]
            gt_file_path = os.path.join(groundtruths_folder_path, filename + '.txt')
            det_file_path = os.path.join(detections_folder_path, filename + '.txt')

            img_path = annotations_list['image_path']

            with open(gt_file_path, 'w') as gt_file:
                for idx in range(num_objs):
                    class_name = test_dataloader.dataset.label_to_name(gt_class[idx]).replace(' ', '_')
                    line = class_name + ' '
                    line += " ".join(str(item) for item in gt_boxes[idx])
                    line += '\n'
                    gt_file.write(line)

              
            keep_boxes, keep_class, keep_score = get_prediction_detection(model_od=model_od, 
                                                                          image=image, 
                                                                          img_size=img_size_od, 
                                                                          threshold=threshold)

            if model_cl is not None:
                img = cv.imread(img_path) # Read image with cv
                img = cv.resize(img, (img_size_od[0], img_size_od[1]))
                keep_score = []
                keep_class = []
                for idx in range(len(keep_boxes)):
                    xmin = int(keep_boxes[idx][0])
                    ymin = int(keep_boxes[idx][1])
                    xmax = int(keep_boxes[idx][2])
                    ymax = int(keep_boxes[idx][3])

                    img_cropped = img[ymin: ymax, xmin: xmax]

                    class_id, score = get_prediction_classification(model_cl, img_cropped, img_size_cl, device)

                    keep_class.append(class_id)
                    keep_score.append(score)

            with open(det_file_path, 'w') as det_file:
                for idx in range(len(keep_boxes)):
                    class_name = test_dataloader.dataset.label_to_name(keep_class[idx]).replace(' ', '_')
                    line = class_name + ' '
                    line += str(round(keep_score[idx], 4))
                    line += ' '
                    line += " ".join(str(item) for item in keep_boxes[idx])
                    line += '\n'
                    det_file.write(line)

            # if i == 1:
            #     break

In [9]:
parser = argparse.ArgumentParser(description='Testing script for Car Model Detection')

# testing arg
parser.add_argument('--path_od', default=None, required=True,
                    help='path to object detection model to be tested')
parser.add_argument('--path_cl', default=None, required=False,
                    help='path to classifier model to be tested')
parser.add_argument('--threshold', default=0.5, required=True, type=float,
                    help='detetions with scores >= threshold')
parser.add_argument('--config_dataset', default=None, required=False,
                    help='config to construct dataset, only if classifier is not None')


# model_path_od = 'logs/resnet18_900_15_finetune/1'
# model_path_cl = None
# config_dataset = None

model_path_od = 'logs/train_only_car/resnet18_900_10_feature_extract_multi_anchors/1'
model_path_cl = 'logs/train_only_car/classifier/resnet34_224_50_grad_cam/1'
config_dataset = 'logs/resnet18_900_15_finetune/1/config.json'


args = parser.parse_args(args=['--path_od', model_path_od, 
                               '--path_cl', model_path_cl, 
                               '--config_dataset', config_dataset,
                               '--threshold', '0.5'])

main(args)

Initializing dataset and dataloader VOC for test...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.


Loading model for car classification....
resnet34


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth



Loading model for car detection....


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth



Car only anchor
Multi anchors
1/3851


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


2/3851
3/3851
4/3851
5/3851
6/3851
7/3851
8/3851
9/3851
10/3851
11/3851
12/3851
13/3851
14/3851
15/3851
16/3851
17/3851
18/3851
19/3851
20/3851
21/3851
22/3851
23/3851
24/3851
25/3851
26/3851
27/3851
28/3851
29/3851
30/3851
31/3851
32/3851
33/3851
34/3851
35/3851
36/3851
37/3851
38/3851
39/3851
40/3851
41/3851
42/3851
43/3851
44/3851
45/3851
46/3851
47/3851
48/3851
49/3851
50/3851
51/3851
52/3851
53/3851
54/3851
55/3851
56/3851
57/3851
58/3851
59/3851
60/3851
61/3851
62/3851
63/3851
64/3851
65/3851
66/3851
67/3851
68/3851
69/3851
70/3851
71/3851
72/3851
73/3851
74/3851
75/3851
76/3851
77/3851
78/3851
79/3851
80/3851
81/3851
82/3851
83/3851
84/3851
85/3851
86/3851
87/3851
88/3851
89/3851
90/3851
91/3851
92/3851
93/3851
94/3851
95/3851
96/3851
97/3851
98/3851
99/3851
100/3851
101/3851
102/3851
103/3851
104/3851
105/3851
106/3851
107/3851
108/3851
109/3851
110/3851
111/3851
112/3851
113/3851
114/3851
115/3851
116/3851
117/3851
118/3851
119/3851
120/3851
121/3851
122/3851
123/3851
124/3851

In [10]:
# # Start tensorboard.
# %reload_ext tensorboard
# %tensorboard --logdir 'logs/train_only_car/resnet18_700_6/1/runs'

In [11]:
# !python Object-Detection-Metrics/pascalvoc.py -gtformat 'xyrb' -detformat 'xyrb'

In [12]:
# %%shell

# cat mAP/input/groundtruths/000572.txt

In [13]:
# %%shell

# python mAP/main.py -na

In [14]:
%%shell

python Object-Detection-Metrics/pascalvoc.py -gt '/content/gdrive/My Drive/licenta/car_model_od/logs/train_only_car/classifier/resnet34_224_50_grad_cam/1/groundtruths' \
    -det '/content/gdrive/My Drive/licenta/car_model_od/logs/train_only_car/classifier/resnet34_224_50_grad_cam/1/detections' \
    -sp '/content/gdrive/My Drive/licenta/car_model_od/logs/train_only_car/classifier/resnet34_224_50_grad_cam/1/results' \
    -gtformat 'xyrb' -detformat 'xyrb'

Streaming output truncated to the last 5000 lines.
012643.txt
012644.txt
012652.txt
012653.txt
012658.txt
012659.txt
012660.txt
012662.txt
012664.txt
012665.txt
012675.txt
012678.txt
012689.txt
012694.txt
012696.txt
012697.txt
012700.txt
012701.txt
012706.txt
012708.txt
012718.txt
012721.txt
012722.txt
012723.txt
012724.txt
012725.txt
012727.txt
012729.txt
012730.txt
012732.txt
012734.txt
012735.txt
012739.txt
012740.txt
012741.txt
012745.txt
012749.txt
012750.txt
012751.txt
012762.txt
012767.txt
012769.txt
012770.txt
012775.txt
012857.txt
012859.txt
012860.txt
012867.txt
012870.txt
012872.txt
012875.txt
012883.txt
012884.txt
012888.txt
012892.txt
012895.txt
012897.txt
012899.txt
012901.txt
012902.txt
012906.txt
012907.txt
012908.txt
012915.txt
012917.txt
012920.txt
012925.txt
012933.txt
012938.txt
012960.txt
012969.txt
012979.txt
012981.txt
012982.txt
012988.txt
012990.txt
012993.txt
012996.txt
013003.txt
013011.txt
013013.txt
013014.txt
013017.txt
013020.txt
013026.txt
013028.txt
013